# Análisis Numérico de EDPs : Ecuación de Parabólicas con Diferencias Finitas

## Ignacio Garach Vélez

## Ecuación de Calor : Euler explícito

$$ \partial_t u = \partial_x^2 u  $$
$$ u(0,x) = u_0(x)  $$
$$ u(t,0) = u(t,1) = 0 $$

Discretizaremos la segunda derivada del mismo modo que habíamos usado hasta ahora y discretizamos en tiempo de forma explícita:

$$ u_{j+1} = u_j + kLu_j$$ donde L es la matriz que usamos en el primer problema con -2 en la diagonal principal y 1 en las adyacentes. Simplemente estamos aproximando en tiempo con 
$$∂_tu_{j+1} = \frac{u_{j+1}-u_j}{k}$$ .

También definimos una función para animar los resultados de los cálculos a lo largo del tiempo.

In [1]:
import numpy as np
import scipy as sp
from scipy.sparse import diags
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import time

In [2]:
def EulerExplicit(f, Np, Mp, t0=0.0, tn=1.0, x0=-1.0, xn=1.0):
  N=Np
  M=Mp

  a=t0
  L=tn
  k = (L-a)/N

  h = (xn - x0)/M

  nodost = [a+k*i for i in np.arange(N+1)]
  nodosx = [x0+h*i for i in np.arange(M+1)]

  inicial = np.array([f(x) for x in nodosx])

  diagonals = [-2*np.ones(M+1), np.ones(M), np.ones(M)]
  mat = (k/h**2)*diags(diagonals, (0, 1, -1)).toarray() + np.identity(M+1)

  mat[0, 0]=1*(k/h**2)+1
  mat[0, 1]=0
  mat[M, M-1]=0
  mat[M, M]=1*(k/h**2)+1

  result=[inicial]
  for i in np.arange(1, N+1):
    result.append(mat@result[i-1])


  return nodost, nodosx, result

from matplotlib import rc
rc('animation', html='jshtml')

def plotAnimation(time, space, solution):
  fig, ax = plt.subplots()
  ax.set_xlabel('x')
  ax.set_ylabel('tª')
  plotLine, = ax.plot(space, np.zeros(len(space))*np.NaN, 'r-')
  plotTitle = ax.set_title("t=0")
  ax.set_xlim(np.min(space)-0.1, np.max(space)+0.1)
  ax.set_ylim(np.min(solution[0]), np.max(solution[0])+0.1)

  def animate(t):
      pp = solution[t]
      plotLine.set_ydata(pp)
      plotTitle.set_text(f"t = {time[t]:.1f}")
      return [plotLine,plotTitle]


  anim = animation.FuncAnimation(fig, func=animate, frames=np.arange(0, len(solution), 1), blit=True)
  return anim
  

Probaremos la implementación con condición inicial la parábola $u_0(x)=-x^2+1$ en el intervalo $[-1, 1]$, observaremos que como hemos estudiado este método numérico con esquema Euler Explícito es inestable para una gran parte del espacio paramétrico (relaciones entre h y k), en la primera elección de nodos es inestable, en la segunda, estamos dentro de la pequeña región de estabilidad.

In [6]:
def parabola(x):
  return -x**2+1

tgridi, xgridi, resultadoi = EulerExplicit(parabola, 20, 10) 
tgrid, xgrid, resultado = EulerExplicit(parabola, 200, 10) 

Mostramos este ejemplo inestable:

In [ ]:
plotAnimation(tgridi, xgridi, resultadoi)

Claramente el anterior no calcula la solución, en la siguiente animación vemos que para una elección de partición dentro de la región de estabilidad sí funciona.

In [ ]:
plotAnimation(tgrid, xgrid, resultado)

Probamos ahora con otra condición inicial $f(x)=sen(2\pi x)$ en $[0, 1]$ que el comportamiento es el mismo, en este caso el comportamiento inestable es interesante por su relación con el estudio de estabilidad que hicimos:

In [12]:
def sin2pix(x):
  return np.sin(2*np.pi*x)

tgridi, xgridi, resultadoi = EulerExplicit(sin2pix, 20, 10, x0=0) 
tgrid, xgrid, resultado = EulerExplicit(sin2pix, 300, 10, x0=0) 

En el caso inestable, al principio si parece que la curva se va achicando pero tiene un cambio de signo en cada iteración, pero llega un punto en el que la inestabilidad pasa a ser de ampliación de magnitud(del estilo a la de la parábola). Esto puede ocurrir porque al principio de la simulación, el valor propio dominante que provoca la inestabilidad provoque el cambio de signo, pero en cierto momento, pase a ser otro valor y su función asociada la que hagan que la inestabilidad cambie de tipo.

In [ ]:
plotAnimation(tgridi, xgridi, resultadoi)

En la región estable no hay problema, no lo mostramos para que no se dispare el tamaño del documento por las animaciones.

## Estabilidad en toda partición: Euler Implícito

Hemos visto que el método anterior tiene problemas de inestabilidad, la solución es el método de Euler implícito, que con la misma aproximación de las derivadas, pero desplazando un instante en tiempo de modo que no basta con producto de matriz por vector para calcular cada paso, ahora deberemos resolver un sistema cada vez.

Este simple cambio nos da un método que es estable en cualquier partición que tomemos.

$$ u_{j+1} = u_j + kLu_{j+1} $$

## Método
$$ u_{j+1} = (I-kL)^{-1} u_j $$


In [15]:
def EulerImplicit(f, Np, Mp, t0=0.0, tn=1.0, x0=-1.0, xn=1.0):
  N=Np
  M=Mp

  a=t0
  L=tn
  k = (L-a)/N

  h = (xn - x0)/M

  nodost = [a+k*i for i in np.arange(N+1)]
  nodosx = [x0+h*i for i in np.arange(M+1)]

  inicial = np.array([f(x) for x in nodosx])

  diagonals = [-2*np.ones(M+1), np.ones(M), np.ones(M)]
  mat = np.identity(M+1) - (k/h**2)*diags(diagonals, (0, 1, -1)).toarray()

  mat[0, 0]=1
  mat[0, 1]=0
  mat[M, M-1]=0
  mat[M, M]=1

  result=[inicial]
  for i in np.arange(1, N+1):
    result.append(np.linalg.inv(mat)@result[i-1])


  return nodost, nodosx, result

In [16]:
tgridi, xgridi, resultadoi = EulerImplicit(sin2pix, 20, 10, x0=0) 
tgrid, xgrid, resultado = EulerImplicit(sin2pix, 300, 10, x0=0) 

Esta simulación, con el método explícito era inestable, ahora es totalmente estable.

In [ ]:
plotAnimation(tgridi, xgridi, resultadoi)

## Esquemas de tipo Crank Nicholson

Vamos a implementar ahora otro método que puede considerarse la media de los otros métodos porque utiliza discretización en diferencias centradas, sin embargo continua siendo implícito y tiene unas condiciones de estabilidad bastante razonables.

## Método

$$ u_{j+1} = u_j + \frac{k}{2}(Lu_j + Lu_{j+1}) $$
$$ u_{j+1} = (I-\frac{k}{2}L)^{-1} (I+\frac{k}{2}L) u_j $$

In [19]:
def CrankNicholson(f, Np, Mp, t0=0.0, tn=1.0, x0=-1.0, xn=1.0):
  N=Np
  M=Mp

  a=t0
  L=tn
  k = (L-a)/N

  h = (xn - x0)/M

  nodost = [a+k*i for i in np.arange(N+1)]
  nodosx = [x0+h*i for i in np.arange(M+1)]

  inicial = np.array([f(x) for x in nodosx])

  diagonals = [-2*np.ones(M+1), np.ones(M), np.ones(M)]
  mat = (k/(2*h**2))*diags(diagonals, (0, 1, -1)).toarray()

  mat[0, 0]=1
  mat[0, 1]=0
  mat[M, M-1]=0
  mat[M, M]=1

  result=[inicial]
  for i in np.arange(1, N+1):
    result.append(np.linalg.inv(np.identity(M+1) - mat) @ (np.identity(M+1) + mat) @ result[i-1])


  return nodost, nodosx, result

## Ecuación de advección

$$ \partial_tu = \partial_xu$$
$$ u(0,x)=u_0(x) $$

con condiciones periódicas:
$$ u(t,0) = u(t,1)$$

## Euler explicito upwind : Estable si k menor o igual que h

$u_{j+1} = u_j + \frac{k}{h}Lu_j$ pero en este caso L tiene -1 en la diagonal, 1 en la adyacente superior y un 1 aislado en la esquina inferior izquierda como condición periódica.


In [23]:
def EulerExplicitAdvection(f, Np, Mp, t0=0.0, tn=10.0, x0=-1.0, xn=1.0):
  N=Np
  M=Mp

  a=t0
  L=tn
  k = (L-a)/N

  h = (xn - x0)/M

  nodost = [a+k*i for i in np.arange(N+1)]
  nodosx = [x0+h*i for i in np.arange(M+1)]

  inicial = np.array([f(x) for x in nodosx])

  diagonals = [-1*np.ones(M+1), np.ones(M), -1*np.zeros(M)]
  mat = (k/(h))*diags(diagonals, (0, 1, -1)).toarray() + np.identity(M+1)

  #mat[0, M]=1
  mat[M, 0]=k/h
  #print(mat)
  result=[inicial]
  for i in np.arange(1, N+1):
    result.append(mat@result[i-1])


  return nodost, nodosx, result

In [ ]:
def cos2pix(x):
  return np.cos(2*np.pi*x)

In [ ]:
tgridi, xgridi, resultadoi = EulerExplicitAdvection(cos2pix, 1000, 100, x0=0, xn=1) 
plotAnimation(tgridi, xgridi, resultadoi)

## Lax Wendroff

$$ u_{j+1, n} = u_{jn} + \frac{k}{2h} (u_{j,n+1}-u_{j,n-1}) + \frac{k^2}{2h^2} (u_{j,n-1}-u_{jn}+u_{j,n-1}) $$

In [29]:
def LaxWendroffAdvection(f, Np, Mp, t0=0.0, tn=10.0, x0=-1.0, xn=1.0):
  N=Np
  M=Mp

  a=t0
  L=tn
  k = (L-a)/N

  h = (xn - x0)/M

  nodost = [a+k*i for i in np.arange(N+1)]
  nodosx = [x0+h*i for i in np.arange(M+1)]

  inicial = np.array([f(x) for x in nodosx])

  diagonals1 = [0*np.ones(M+1), np.ones(M), -1*np.ones(M)]
  diagonals2 = [-2*np.ones(M+1), np.ones(M), np.ones(M)]

  mat1 = (k/(2*h))*diags(diagonals1, (0, 1, -1)).toarray()

  mat2 = (k**2/(2*h**2))*diags(diagonals2, (0, 1, -1)).toarray()

  mat1[0, M]=(k/(2*h))
  mat2[0, M]=(k**2/(2*h**2))
  mat2[M, 0]=(k**2/(2*h**2))
  
  result=[inicial]
  for i in np.arange(1, N+1):
    result.append(result[i-1] + mat1@result[i-1] + mat2@result[i-1])


  return nodost, nodosx, result